## Distortion-Based Analysis of Single Transferable Vote (Code)

This file contains code which I (Vikram Kher) have written during my time researching Single Transferable Vote (STV). The program generates a sample election scenario with c candidates and 2^(c-1) voter profiles. It then constructs and solves a linear program (via the library pulp) to find the distortion of STV in the given election scenario.

### Import Necessary Libraries

Note: pulp is a python LP solver

In [1]:
from pulp import *
import math
from string import ascii_uppercase

In [2]:
# Set up LP maximization problem
model = pulp.LpProblem("Distortion_Maximization_problem", LpMaximize)

### Generate Sample Voter Preference Lists

Note: There are many possible election scenarios which are consistent with the rules of STV; this just represents one scenario which I investigated.

In [3]:
#Choose number of candidates in election
c = 5

In [4]:
alpha = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
voters = []
candidates = []

v = 2**(c-1)

# Create list of voter profiles
for i in range(0, v):
    voters.append(i)


# Create list of candidates
for i in range(0,c): 
    candidates.append(alpha[i])
    
preferenceList = dict()
val = []
for i in range(0,v):
    preferenceList[voters[i]] = []
    val.append(0)
    
    
val[0] = 1/2
val[1] = 1/2
def build(preferenceList, i,j):
    for k in range(i,v):
        if(k >= 2**j):
            j = j + 1
        preferenceList[voters[k]].append(candidates[j])
        for q in range(0,c-j-1):
            preferenceList[voters[k]].append(candidates[c-q-1])
    
    build(preferenceList,i+1,j+1)
        
def rebuild(preferenceList, n):
    k = 4
    p = 2
    preferenceList[0].append(candidates[0])
    preferenceList[0].append(candidates[1])
    preferenceList[1].append(candidates[1])
    preferenceList[1].append(candidates[0])
    divide(preferenceList, val, k, p)
                

def divide(preferenceList,val, k, p):
    if k < 2**c-1:
        for i in range(0,int(k/2)):
            #Duplicate preferences
            preferenceList[i].insert(1,candidates[p])
            preferenceList[k/2+i] = preferenceList[i].copy()
            
            #Swap Candidates
            temp = preferenceList[k/2+i][0]
            preferenceList[k/2+i][0] = preferenceList[k/2+i][1]
            preferenceList[k/2+i][1] = temp
            
            #Split frequencies
            val[int(k/2)+i] = val[i]/(p+1)
            val[i]= val[i] - val[int(k/2)+i]
        
        divide(preferenceList, val, k*2,p+1)

rebuild(preferenceList, c)

In [5]:
#Prints voter profiles
print(preferenceList)

{0: ['A', 'E', 'D', 'C', 'B'], 1: ['B', 'E', 'D', 'C', 'A'], 2: ['C', 'E', 'D', 'A', 'B'], 3: ['C', 'E', 'D', 'B', 'A'], 4: ['D', 'E', 'A', 'C', 'B'], 5: ['D', 'E', 'B', 'C', 'A'], 6: ['D', 'E', 'C', 'A', 'B'], 7: ['D', 'E', 'C', 'B', 'A'], 8: ['E', 'A', 'D', 'C', 'B'], 9: ['E', 'B', 'D', 'C', 'A'], 10: ['E', 'C', 'D', 'A', 'B'], 11: ['E', 'C', 'D', 'B', 'A'], 12: ['E', 'D', 'A', 'C', 'B'], 13: ['E', 'D', 'B', 'C', 'A'], 14: ['E', 'D', 'C', 'A', 'B'], 15: ['E', 'D', 'C', 'B', 'A']}


In [6]:
#Prints frequency of voter profiles amoung voter population
print(val)

[0.2, 0.2, 0.1, 0.1, 0.06666666666666668, 0.06666666666666668, 0.03333333333333333, 0.03333333333333333, 0.05, 0.05, 0.025, 0.025, 0.01666666666666667, 0.01666666666666667, 0.008333333333333333, 0.008333333333333333]


In [7]:
# Prints labels of candidates in election
candidates

['A', 'B', 'C', 'D', 'E']

In [8]:
# Prints labels of voters in election
voters

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

### Add Variables to LP

In [9]:
voter_candidate_pairs = pulp.LpVariable.dicts("pair",
                                     ((i,j) for i in voters for j in candidates),
                                     lowBound=0,
                                     cat='Continuous')

In [10]:
voter_candidate_pairs

{(0, 'A'): pair_(0,_'A'),
 (0, 'B'): pair_(0,_'B'),
 (0, 'C'): pair_(0,_'C'),
 (0, 'D'): pair_(0,_'D'),
 (0, 'E'): pair_(0,_'E'),
 (1, 'A'): pair_(1,_'A'),
 (1, 'B'): pair_(1,_'B'),
 (1, 'C'): pair_(1,_'C'),
 (1, 'D'): pair_(1,_'D'),
 (1, 'E'): pair_(1,_'E'),
 (2, 'A'): pair_(2,_'A'),
 (2, 'B'): pair_(2,_'B'),
 (2, 'C'): pair_(2,_'C'),
 (2, 'D'): pair_(2,_'D'),
 (2, 'E'): pair_(2,_'E'),
 (3, 'A'): pair_(3,_'A'),
 (3, 'B'): pair_(3,_'B'),
 (3, 'C'): pair_(3,_'C'),
 (3, 'D'): pair_(3,_'D'),
 (3, 'E'): pair_(3,_'E'),
 (4, 'A'): pair_(4,_'A'),
 (4, 'B'): pair_(4,_'B'),
 (4, 'C'): pair_(4,_'C'),
 (4, 'D'): pair_(4,_'D'),
 (4, 'E'): pair_(4,_'E'),
 (5, 'A'): pair_(5,_'A'),
 (5, 'B'): pair_(5,_'B'),
 (5, 'C'): pair_(5,_'C'),
 (5, 'D'): pair_(5,_'D'),
 (5, 'E'): pair_(5,_'E'),
 (6, 'A'): pair_(6,_'A'),
 (6, 'B'): pair_(6,_'B'),
 (6, 'C'): pair_(6,_'C'),
 (6, 'D'): pair_(6,_'D'),
 (6, 'E'): pair_(6,_'E'),
 (7, 'A'): pair_(7,_'A'),
 (7, 'B'): pair_(7,_'B'),
 (7, 'C'): pair_(7,_'C'),
 (7, 'D'): p

### Add objective function

In [11]:
model += (
    pulp.lpSum([voter_candidate_pairs[(j, candidates[0])]*val[i] for i,j in enumerate(voters)])
)
model

Distortion_Maximization_problem:
MAXIMIZE
0.2*pair_(0,_'A') + 0.2*pair_(1,_'A') + 0.025*pair_(10,_'A') + 0.025*pair_(11,_'A') + 0.01666666666666667*pair_(12,_'A') + 0.01666666666666667*pair_(13,_'A') + 0.008333333333333333*pair_(14,_'A') + 0.008333333333333333*pair_(15,_'A') + 0.1*pair_(2,_'A') + 0.1*pair_(3,_'A') + 0.06666666666666668*pair_(4,_'A') + 0.06666666666666668*pair_(5,_'A') + 0.03333333333333333*pair_(6,_'A') + 0.03333333333333333*pair_(7,_'A') + 0.05*pair_(8,_'A') + 0.05*pair_(9,_'A') + 0.0
VARIABLES
pair_(0,_'A') Continuous
pair_(1,_'A') Continuous
pair_(10,_'A') Continuous
pair_(11,_'A') Continuous
pair_(12,_'A') Continuous
pair_(13,_'A') Continuous
pair_(14,_'A') Continuous
pair_(15,_'A') Continuous
pair_(2,_'A') Continuous
pair_(3,_'A') Continuous
pair_(4,_'A') Continuous
pair_(5,_'A') Continuous
pair_(6,_'A') Continuous
pair_(7,_'A') Continuous
pair_(8,_'A') Continuous
pair_(9,_'A') Continuous

### Add Constraints

#### Triangle Inequality

In [12]:
for i in voters:
    for j in candidates:
        for k in voters:
            for l in candidates:
                model += voter_candidate_pairs[(i,j)] <= voter_candidate_pairs[(k,j)] + voter_candidate_pairs[(k,l)] + voter_candidate_pairs[(i,l)]
model

Distortion_Maximization_problem:
MAXIMIZE
0.2*pair_(0,_'A') + 0.2*pair_(1,_'A') + 0.025*pair_(10,_'A') + 0.025*pair_(11,_'A') + 0.01666666666666667*pair_(12,_'A') + 0.01666666666666667*pair_(13,_'A') + 0.008333333333333333*pair_(14,_'A') + 0.008333333333333333*pair_(15,_'A') + 0.1*pair_(2,_'A') + 0.1*pair_(3,_'A') + 0.06666666666666668*pair_(4,_'A') + 0.06666666666666668*pair_(5,_'A') + 0.03333333333333333*pair_(6,_'A') + 0.03333333333333333*pair_(7,_'A') + 0.05*pair_(8,_'A') + 0.05*pair_(9,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(0,_'D') <= 0

_C5: 0 pair_(0,_'A') - 2 pair_(0,_'E') <= 0

_C6: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C7: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C8: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C9: pair_(0,_'A') - pair_(0,_'D') - pair_(1,_'A') - pair_(1,_'D') <= 0

_C10: pa

#### Consistency

In [13]:
for i in voters:
    for j in candidates:
        for k in candidates:
            if(preferenceList[i].index(j) < preferenceList[i].index(k)):
                model += voter_candidate_pairs[(i,j)] <= voter_candidate_pairs[(i,k)]
            
model

Distortion_Maximization_problem:
MAXIMIZE
0.2*pair_(0,_'A') + 0.2*pair_(1,_'A') + 0.025*pair_(10,_'A') + 0.025*pair_(11,_'A') + 0.01666666666666667*pair_(12,_'A') + 0.01666666666666667*pair_(13,_'A') + 0.008333333333333333*pair_(14,_'A') + 0.008333333333333333*pair_(15,_'A') + 0.1*pair_(2,_'A') + 0.1*pair_(3,_'A') + 0.06666666666666668*pair_(4,_'A') + 0.06666666666666668*pair_(5,_'A') + 0.03333333333333333*pair_(6,_'A') + 0.03333333333333333*pair_(7,_'A') + 0.05*pair_(8,_'A') + 0.05*pair_(9,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(0,_'D') <= 0

_C5: 0 pair_(0,_'A') - 2 pair_(0,_'E') <= 0

_C6: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C7: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C8: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C9: pair_(0,_'A') - pair_(0,_'D') - pair_(1,_'A') - pair_(1,_'D') <= 0

_C10: pa

#### Normalization

In [14]:
model += pulp.lpSum([voter_candidate_pairs[i, candidates[-1]]*val[voters.index(i)] for i in voters]) == 1
model

Distortion_Maximization_problem:
MAXIMIZE
0.2*pair_(0,_'A') + 0.2*pair_(1,_'A') + 0.025*pair_(10,_'A') + 0.025*pair_(11,_'A') + 0.01666666666666667*pair_(12,_'A') + 0.01666666666666667*pair_(13,_'A') + 0.008333333333333333*pair_(14,_'A') + 0.008333333333333333*pair_(15,_'A') + 0.1*pair_(2,_'A') + 0.1*pair_(3,_'A') + 0.06666666666666668*pair_(4,_'A') + 0.06666666666666668*pair_(5,_'A') + 0.03333333333333333*pair_(6,_'A') + 0.03333333333333333*pair_(7,_'A') + 0.05*pair_(8,_'A') + 0.05*pair_(9,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(0,_'D') <= 0

_C5: 0 pair_(0,_'A') - 2 pair_(0,_'E') <= 0

_C6: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C7: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C8: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C9: pair_(0,_'A') - pair_(0,_'D') - pair_(1,_'A') - pair_(1,_'D') <= 0

_C10: pa

#### Optimality

In [15]:
for i in candidates:
    model += pulp.lpSum([voter_candidate_pairs[j, i]*val[voters.index(j)] for j in voters]) >= 1
model

Distortion_Maximization_problem:
MAXIMIZE
0.2*pair_(0,_'A') + 0.2*pair_(1,_'A') + 0.025*pair_(10,_'A') + 0.025*pair_(11,_'A') + 0.01666666666666667*pair_(12,_'A') + 0.01666666666666667*pair_(13,_'A') + 0.008333333333333333*pair_(14,_'A') + 0.008333333333333333*pair_(15,_'A') + 0.1*pair_(2,_'A') + 0.1*pair_(3,_'A') + 0.06666666666666668*pair_(4,_'A') + 0.06666666666666668*pair_(5,_'A') + 0.03333333333333333*pair_(6,_'A') + 0.03333333333333333*pair_(7,_'A') + 0.05*pair_(8,_'A') + 0.05*pair_(9,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(0,_'D') <= 0

_C5: 0 pair_(0,_'A') - 2 pair_(0,_'E') <= 0

_C6: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C7: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C8: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C9: pair_(0,_'A') - pair_(0,_'D') - pair_(1,_'A') - pair_(1,_'D') <= 0

_C10: pa

#### Positive Distances

Already Done

### Solve Model

In [16]:
model.solve()

1

In [17]:
# Print Distortion
print(pulp.value(model.objective))

3.36363633


In [18]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [19]:
for v in model.variables():
    if(v.name[10] == 'A' or v.name[11] == 'A'):
        print(v.name, "=", v.varValue)

pair_(0,_'A') = 1.8181818
pair_(1,_'A') = 4.5454545
pair_(10,_'A') = 2.7272727
pair_(11,_'A') = 3.6363636
pair_(12,_'A') = 2.7272727
pair_(13,_'A') = 3.6363636
pair_(14,_'A') = 2.7272727
pair_(15,_'A') = 3.6363636
pair_(2,_'A') = 2.7272727
pair_(3,_'A') = 4.5454545
pair_(4,_'A') = 2.7272727
pair_(5,_'A') = 4.5454545
pair_(6,_'A') = 2.7272727
pair_(7,_'A') = 4.5454545
pair_(8,_'A') = 2.7272727
pair_(9,_'A') = 3.6363636


### Extract dual variable values

In [20]:
for name, c in list(model.constraints.items()):
    # Print non-negative dual variables
    if(c.pi != 0):
        print(name, ":", c, "\t Dual", c.pi)

_C405 : -pair_(0,_'A') - pair_(0,_'E') + pair_(1,_'A') - pair_(1,_'E') <= 0 	 Dual 0.065151515
_C415 : pair_(1,_'A') - pair_(1,_'E') - pair_(2,_'A') - pair_(2,_'E') <= 0 	 Dual 0.049242424
_C435 : pair_(1,_'A') - pair_(1,_'E') - pair_(6,_'A') - pair_(6,_'E') <= 0 	 Dual 0.017424242
_C445 : pair_(1,_'A') - pair_(1,_'E') - pair_(8,_'A') - pair_(8,_'E') <= 0 	 Dual 0.038636364
_C465 : pair_(1,_'A') - pair_(1,_'E') - pair_(12,_'A') - pair_(12,_'E') <= 0 	 Dual 0.01969697
_C475 : pair_(1,_'A') - pair_(1,_'E') - pair_(14,_'A') - pair_(14,_'E') <= 0 	 Dual 0.0098484848
_C890 : -pair_(1,_'B') - pair_(1,_'E') + pair_(2,_'B') - pair_(2,_'E') <= 0 	 Dual 0.11287879
_C930 : pair_(2,_'B') - pair_(2,_'E') - pair_(9,_'B') - pair_(9,_'E') <= 0 	 Dual 0.036363636
_C1205 : -pair_(0,_'A') - pair_(0,_'E') + pair_(3,_'A') - pair_(3,_'E') <= 0 	 Dual 0.021212121
_C1225 : pair_(3,_'A') - pair_(3,_'E') - pair_(4,_'A') - pair_(4,_'E') <= 0 	 Dual 0.078787879
_C1775 : -pair_(2,_'C') - pair_(2,_'E') + pair_(4,_'